#### classifying cifar10 dataset using feedforward neural net

In [1]:
#import everything
import torch
import torchvision
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.data import random_split

In [2]:
dataset = CIFAR10(root='datasets/',download=True,transform=ToTensor())

Extracting datasets/cifar-10-python.tar.gz to datasets/


In [3]:
len(dataset)

50000

In [4]:
torch.manual_seed(43) #42 is answer to life universe and everything and seed should be prime so +1

In [12]:
train_dataset,valid_dataset = random_split(dataset,[45000,5000])

In [13]:
test_dataset = CIFAR10(root='datasets/',train=False,transform=ToTensor())

In [14]:
len(test_dataset)

10000

In [8]:
#for GPU optims
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


In [9]:
def to_device(data,device):
    if isinstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device,non_blocking=True) #I don't fully understand this non_blocking term, just saw it somewhere

In [10]:
class DeviceDataLoader(): #wrapper over DataLoader class, to push it to device and create __iter__ class for taking full advantage of GPU
    def __init__(self,data_loader,device):
        self.data_loader = data_loader
        self.device = device
    
    def __iter__(self): #this class makes the magic of GPU happen
        for batch in self.data_loader:
            yield(to_device(batch,self.device))
    
    def __len__(self):
        return len(self.data_loader)

In [11]:
device = get_default_device()
device

device(type='cuda')

In [15]:
#create loaders
batch_size=128
train_loader = DeviceDataLoader(DataLoader(train_dataset,batch_size,shuffle=True,pin_memory=True,num_workers=4),device)
valid_loader = DeviceDataLoader(DataLoader(train_dataset,batch_size*2,pin_memory=True,num_workers=4),device)
test_loader  = DeviceDataLoader(DataLoader(train_dataset,batch_size*2,pin_memory=True,num_workers=4),device)

In [19]:
#model class 
class CIFAR10Model(nn.Module):
    def __init__(self,in_size,num_classes):
        super().__init__()
        self.linear1 = nn.Linear(in_size,16)
        self.linear2 = nn.Linear(16,32)
        self.linear3 = nn.Linear(32,num_classes)
    
    def forward(self,X_batch):
        out = X_batch.view(X_batch.size(0),-1)
        out = self.linear1(out)
        out = F.relu(out)
        out = self.linear2(out)
        out = F.relu(out)
        out = self.linear3(out)
        return out
    
    def train_step(self,X_batch):
        images,labels = X_batch
        out = self.forward(images)
        loss = F.cross_entropy(out,labels)
        return loss
    
    def valid_step(self,X_batch):
        images,labels = X_batch
        out = self.forward(images)
        loss = F.cross_entropy(out,labels)
        _,preds = torch.max(out,dim=1)
        acc = torch.tensor(torch.sum(preds==labels).item()/len(preds))
        return {'loss':loss,'acc':acc}

In [17]:
def evaluate(model,loader):
    out = [model.valid_step(batch) for batch in loader]
    batch_loss = [x['loss'] for x in out]
    batch_acc  = [x['acc'] for x in out]
    epoch_loss = torch.stack(batch_loss).mean()
    epoch_acc  = torch.stack(batch_acc).mean()
    print(f"loss is {epoch_loss} and acc is {epoch_acc}")

In [18]:
def train(model,train_loader,valid_loader,num_epochs=10,lr=0.01,optim_func=torch.optim.SGD):
    optim = optim_func(model.parameters(),lr)
    for epoch in range(num_epochs):
        for batch in train_loader:
            loss = model.train_step(batch)
            loss.backward()
            optim.step()
            optim.zero_grad()
        evaluate(model,valid_loader)

In [34]:
model = CIFAR10Model(3*32*32,10)

In [35]:
model = to_device(model,device)

In [36]:
evaluate(model,valid_loader)

loss is 2.3169798851013184 and acc is 0.1039692834019661


In [37]:
train(model,train_loader,valid_loader,20,0.01)

loss is 2.1425089836120605 and acc is 0.17021217942237854
loss is 1.991416335105896 and acc is 0.258444607257843
loss is 1.966501235961914 and acc is 0.27538618445396423
loss is 1.9150358438491821 and acc is 0.3040003478527069
loss is 1.8939666748046875 and acc is 0.31960582733154297
loss is 1.8701751232147217 and acc is 0.3294282555580139
loss is 1.8553366661071777 and acc is 0.33513760566711426
loss is 1.8219321966171265 and acc is 0.34993162751197815
loss is 1.8105708360671997 and acc is 0.3525923490524292
loss is 1.800219178199768 and acc is 0.35188475251197815
loss is 1.7753745317459106 and acc is 0.3679882884025574
loss is 1.760054349899292 and acc is 0.3707972466945648
loss is 1.75911545753479 and acc is 0.36866655945777893
loss is 1.7283658981323242 and acc is 0.3862091302871704
loss is 1.73177969455719 and acc is 0.3816849887371063
loss is 1.7058794498443604 and acc is 0.39375531673431396
loss is 1.6966466903686523 and acc is 0.3955557644367218
loss is 1.7286349534988403 and a

In [38]:
train(model,train_loader,valid_loader,10,0.01)

loss is 1.697308897972107 and acc is 0.4004066288471222
loss is 1.6716159582138062 and acc is 0.405342698097229
loss is 1.6825312376022339 and acc is 0.39685899019241333
loss is 1.6554441452026367 and acc is 0.4097975790500641
loss is 1.7212398052215576 and acc is 0.3892178535461426
loss is 1.663379430770874 and acc is 0.40922051668167114
loss is 1.6307307481765747 and acc is 0.4199378788471222
loss is 1.629482626914978 and acc is 0.4206126034259796
loss is 1.632980227470398 and acc is 0.41924893856048584
loss is 1.6295881271362305 and acc is 0.41804152727127075


In [39]:
train(model,train_loader,valid_loader,2,0.001)

loss is 1.5931857824325562 and acc is 0.43644973635673523
loss is 1.5921663045883179 and acc is 0.4371280372142792


In [40]:
train(model,train_loader,valid_loader,10,0.001)

loss is 1.5921789407730103 and acc is 0.4377654790878296
loss is 1.590474247932434 and acc is 0.43742629885673523
loss is 1.5900518894195557 and acc is 0.43892577290534973
loss is 1.589621901512146 and acc is 0.4382093548774719
loss is 1.588780164718628 and acc is 0.43806639313697815
loss is 1.5883747339248657 and acc is 0.4387455880641937
loss is 1.5880250930786133 and acc is 0.4377468228340149
loss is 1.5870319604873657 and acc is 0.43868163228034973
loss is 1.5874300003051758 and acc is 0.43840286135673523
loss is 1.587885856628418 and acc is 0.4376482665538788
